# Wildfire Analysis
## Michael Peters and Nick Latham
CPSC322
04.06.21

In [45]:
import geojson
import datetime

In [46]:
def load_data(filename):
    """
    Loads GeoJson Data from "filename"
    """
    with open(filename) as f:
        data = geojson.load(f)
    return data

In [47]:
older_fire_data = load_data("data/DNR_Fire_Statistics_1970-2007.geojson")
newer_fire_data = load_data("data/DNR_Fire_Statistics_2008_-_Present.geojson")

In [48]:
old_fire_data = older_fire_data["features"]
new_fire_data = newer_fire_data["features"]

In [49]:
print(old_fire_data[1])

{"geometry": {"coordinates": [-117.078338, 47.837034], "type": "Point"}, "properties": {"ACRES_BURNED": 0.1, "COUNTY_LABEL_NM": "SPOKANE", "FIREEVNT_CLASS_LABEL_NM": "Classified", "FIREEVNT_NM": null, "FIREGCAUSE_LABEL_NM": "Recreation", "OBJECTID": 2, "START_DT": "1974-10-12T08:00:00Z"}, "type": "Feature"}


In [50]:
print(new_fire_data[1])

{"geometry": {"coordinates": [-120.916389, 45.904957], "type": "Point"}, "properties": {"ACRES_BURNED": 0.25, "BURNESCAPE_RSN_LABEL_NM": "Extinguish", "BURN_MERCH_AREA": null, "BURN_NONSTOCK_AREA": 0.25, "BURN_REPROD_AREA": null, "CONTROL_DT": "2017-05-23T07:00:00Z", "CONTROL_TM": "1935", "COUNTY_LABEL_NM": "KLICKITAT", "DSCVR_DT": "2017-05-23T07:00:00Z", "DSCVR_TM": "1650", "FIREEVENT_ID": 50035, "FIREEVNT_CLASS_CD": 1, "FIREEVNT_CLASS_LABEL_NM": "Classified", "FIREGCAUSE_LABEL_NM": "Debris Burn", "FIRESCAUSE_LABEL_NM": "None", "FIRE_OUT_DT": "2017-05-25T07:00:00Z", "FIRE_OUT_TM": "1300", "FIRE_RGE_DIR_FLG": "E", "FIRE_RGE_FRACT_NO": 0, "FIRE_RGE_WHOLE_NO": 15, "FIRE_SECT_NO": 22, "FIRE_TWP_FRACT_NO": 0, "FIRE_TWP_WHOLE_NO": 5, "INCIDENT_ID": 49868, "INCIDENT_NM": "Turkey Ranch", "INCIDENT_NO": 7, "LAT_COORD": 45.904947, "LON_COORD": -120.916377, "NON_DNR_RES_ORDER_NO": null, "OBJECTID": 2, "PROTECTION_TYPE": "DNR Protection FFPA", "REGION_NAME": "SOUTHEAST", "RES_ORDER_NO": "WA-SES-0

In [51]:

# get total number of fires
old_total = len(old_fire_data)
new_total = len(new_fire_data)
print(old_total, new_total, old_total + new_total)

38116 20829 58945


In [52]:
def parse_ISO8601(date):
    
    return date

In [53]:
def binLat(lat):
    if lat > 48:
        return 1
    elif 48 >= lat > 47:
        return 2
    elif 47 >= lat > 46:
        return 3
    elif 46 >= lat > 45:
        return 4
    else:
        return 5

def binLon(lon):
    if lon < -124:
        return 1
    elif -124 <= lon < -123:
        return 2
    elif -123 <= lon < -122:
        return 3
    elif -122 <= lon < -121:
        return 4
    elif -121 <= lon < -120:
        return 5
    elif -120 <= lon < -119:
        return 6
    elif -119 <= lon < -118:
        return 7
    else:
        return 8

def binAcres(acres):
    if acres < 2:
        return 1
    elif 2 <= acres < 10:
        return 2
    elif 10 <= acres < 50:
        return 3
    elif 50 <= acres < 100:
        return 4
    elif 100 <= acres < 500:
        return 5
    elif 500 <= acres < 2000:
        return 6
    elif 2000 <= acres < 10000:
        return 7
    elif 10000 <= acres < 50000:
        return 8
    else:
        return 9

In [54]:
cleaned_fire_data = []
counties = []
causes = []
for fire in old_fire_data:
    date = fire["properties"]["START_DT"]
    acres = fire["properties"]["ACRES_BURNED"]
    county = fire["properties"]["COUNTY_LABEL_NM"]
    cause = fire["properties"]["FIREGCAUSE_LABEL_NM"]
    lon = fire["geometry"]["coordinates"][0]
    lat = fire["geometry"]["coordinates"][1]
    binlon = binLon(lon)
    binlat = binLat(lat)
    binacres = binAcres(acres)
    date = datetime.datetime.strptime(date.split("-")[1], "%m").strftime("%b")
    cleaned_fire_data.append([date, acres, county, cause, lat, lon, binlat, binlon, binacres])
    counties.append(county)
    causes.append(cause)

for fire in new_fire_data: 
    date = fire["properties"]["START_DT"]
    acres = fire["properties"]["ACRES_BURNED"]
    county = fire["properties"]["COUNTY_LABEL_NM"]
    cause = fire["properties"]["FIREGCAUSE_LABEL_NM"]
    lon = fire["geometry"]["coordinates"][0]
    lat = fire["geometry"]["coordinates"][1]
    binlon = binLon(lon)
    binlat = binLat(lat)
    binacres = binAcres(acres)
    date = datetime.datetime.strptime(date.split("-")[1], "%m").strftime("%b")
    cleaned_fire_data.append([date, acres, county, cause, lat, lon, binlat, binlon, binacres])
    counties.append(county)
    causes.append(cause)

In [55]:
# remove fires smaller than X acres
trimmed_fire_data = []
X = 5

print(len(cleaned_fire_data))
for i, fire in enumerate(cleaned_fire_data):
    if fire[1] > X:
        trimmed_fire_data.append(fire)

print(len(trimmed_fire_data))

58945
3737


In [56]:
import csv

In [57]:
with open('data/clean_fire_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile,)
    writer.writerow(("date", "acres", "county" , "cause", "lat", "lon", "binlat", "binlon", "binacres"))
    for fire in trimmed_fire_data:
        writer.writerow(fire)

In [58]:
uniq_county = set(counties)
uniq_cause = set(causes)

print(uniq_county)
print(uniq_cause)

{'NO DATA', 'OKANOGAN', 'WHATCOM', 'THURSTON', 'KITSAP', 'FERRY', 'YAKIMA', 'SAN JUAN', 'JEFFERSON', 'LEWIS', 'DOUGLAS', 'SKAGIT', None, 'SKAMANIA', 'GARFIELD', 'KING', 'COWLITZ', 'BENTON', 'LINCOLN', 'CHELAN', 'ADAMS', 'GRANT', 'PEND OREILLE', 'SNOHOMISH', 'FRANKLIN', 'CLARK', 'STEVENS', 'KITTITAS', 'ASOTIN', 'PIERCE', 'GRAYS HARBOR', 'KLICKITAT', 'WALLA WALLA', 'PACIFIC', 'CLALLAM', 'SPOKANE', 'WAHKIAKUM', 'WHITMAN', 'COLUMBIA', 'ISLAND', 'MASON'}
{'Children', 'Arson', 'None', 'Undetermined', 'Logging', 'Railroad', 'Lightning', 'Miscellaneou', 'Under Invest', 'Recreation', 'Debris Burn', 'Smoker'}
